###### guess what I ended up doing is exploring the data structure a little more. 
The forecasting works to the individual level, which is cool, but not useable...  16,000 upcs .  So I wanted to 'add' a little bit of a 'scope' for the data (not saying its correct).  
Data comes out cool in excel, easy pivot table,  pivoted in python too,  probably all doable in Quicksight for this part but I wanted to see what it looked like when grouping up to Category and Sub Department.  Also asked it to reduct to only items totals that made up 50% of sales. 

In [39]:
import pandas as pd
import numpy as np
import random
import sys
import os

sys.path.append(os.path.abspath('..'))
from utils.db import get_db

In [46]:
engine = get_db()
start_date = '5/05/2025'
end_date = '6/23/2025'
store_number = '440'

In [47]:
query = f"""SELECT 
    storeid,
    sale_date,
    product_code,
    product_description,
    category,
    category_description,
    sub_department,
    sub_department_description,
    price,
    cost,
    case_size,
    qty,
    total_sales, 
    sale_type
FROM sales
WHERE sale_date BETWEEN '{start_date}' and '{end_date}'
  AND store_number IN ('{store_number}')
"""

In [48]:
ds = pd.read_sql(query, engine)

#ensure it is datetime format

ds["sale_date"] = pd.to_datetime(ds["sale_date"])

2025-06-30 17:00:55,048 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-06-30 17:00:55,049 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-30 17:00:55,104 INFO sqlalchemy.engine.Engine select current_schema()
2025-06-30 17:00:55,105 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-30 17:00:55,161 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-06-30 17:00:55,162 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-30 17:00:55,235 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-30 17:00:55,236 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

In [49]:
ds = ds.dropna(subset=['price', 'cost', 'qty','case_size'])
ds = ds[ds['qty'] > 0]

ds["cost"] = (ds["cost"])/(ds["case_size"])
ds.head(10)

,storeid,sale_date,product_code,product_description,category,category_description,sub_department,sub_department_description,price,cost,case_size,qty,total_sales,sale_type
20,1397,2025-06-10,4011.0,BANANA LB,84000.0,BANANAS,25.0,Wic Produce,0.69,0.010000,1.0,1.0,0.88,Sale
21,1397,2025-06-10,4011.0,BANANA LB,84000.0,BANANAS,25.0,Wic Produce,0.69,0.010000,1.0,1.0,1.57,Sale
60,1397,2025-06-09,1112017846.0,CARPET CLEANER 48 HOUR RENTAL N/A,17998.0,RUG DOCTOR,3.0,Nonfood,49.99,22.500000,1.0,1.0,49.99,Sale
64,1397,2025-06-09,8290122838.0,9`X12` 3 MIL DROPCLOTH,814139.0,DROP CLOTHS (ACE),814.0,Paint and Paint Supplies,8.99,2.910000,12.0,1.0,8.99,Sale
65,1397,2025-06-09,8304600402.0,ICE MOUNTAIN 16.9 OZ 24 PK PET 405.6 OZ,36005.0,WATER PLASTIC BOTTLE,1.0,Grocery,5.99,5.540000,1.0,1.0,4.99,Sale
66,1397,2025-06-09,730.0,FRESH ARRANGEMENT OR STEMS,130020.0,ARRANGEMENTS,29.0,Floral,49.99,0.010000,1.0,1.0,49.99,Sale
67,1397,2025-06-09,9477608187.0,FRUIT SLICES 17 OZ,90020.0,POPCORN AND DRIED FRUIT,11.0,Produce,5.39,2.211667,12.0,1.0,5.39,Sale
70,1397,2025-06-09,28000500000.0,GROUND BEEF GROUND ROUND LB,90201.0,Fresh Meat Ground Beef,21.0,Meat by #,6.99,0.010000,1.0,1.0,5.91,Sale
71,1397,2025-06-09,28000600000.0,FRESH GROUND CHUCK 80/20 LB,90201.0,Fresh Meat Ground Beef,21.0,Meat by #,5.99,0.010000,1.0,1.0,9.99,Sale
72,1397,2025-06-09,28140800000.0,PR BNLS CHCK RST EA,90141.0,BEEF ROASTS,9.0,Meat,11.75,0.010000,1.0,1.0,11.75,Sale


In [50]:
# --- AGGREGATE SALES BY PRODUCT (including sub-department) ---
product_summary = (
    ds.groupby(['product_code', 'category', 'category_description',
                'sub_department', 'sub_department_description'], as_index=False)
    .agg(total_sales=('total_sales', 'sum'),
         total_qty=('qty', 'sum'),
         total_cost=('cost', 'sum'))  # Sum of cost (per sale row)
)

In [51]:
# --- SORT BY SALES DESCENDING ---
product_summary.sort_values(by='total_sales', ascending=False, inplace=True)

In [52]:
# --- CALCULATE CUMULATIVE SALES % ---
product_summary['cumulative_sales'] = product_summary['total_sales'].cumsum()
total_sales_sum = product_summary['total_sales'].sum()
product_summary['cumulative_sales_pct'] = product_summary['cumulative_sales'] / total_sales_sum


In [53]:
# --- FILTER TOP 50% OF SALES CONTRIBUTORS ---
top_50_products = product_summary[product_summary['cumulative_sales_pct'] <= 0.5]

In [37]:
# --- AGGREGATE BACK TO CATEGORY LEVEL ---
category_trends = (
    top_50_products.groupby(['category', 'category_description'], as_index=False)
    .agg(total_sales=('total_sales', 'sum'),
         total_qty=('total_qty', 'sum'),
         distinct_products=('product_code', 'nunique'))
)

In [38]:
# --- SORT BY SALES ---
category_trends = category_trends.sort_values(by='total_sales', ascending=False)

In [23]:
category_trends.head(10)

,category,category_description,total_sales,total_qty,distinct_products
202,999996.0,DELI MISC,65423.03,16117.0,78
132,90201.0,Fresh Meat Ground Beef,47039.62,4223.0,14
38,36099.0,MISC WATER SOFT DRINKS MIXES,38447.40,9227.0,33
201,999995.0,DELI HOT MISC,34566.33,3821.0,15
179,220003.0,MILK,21696.79,5934.0,14
30,33022.0,POTATO CHIP - BAGGED,16531.25,4023.0,30
139,90401.0,Fresh Meat Chicken,16464.37,1989.0,14
157,91601.0,Smoked Meat Bacon,13520.37,2816.0,19
129,90121.0,BEEF GRILLING STEAKS,12371.91,1168.0,9
128,90101.0,Fresh Meat Beef,11015.06,1477.0,12


In [14]:
# --- OPTIONAL EXPORT ---
category_trends.to_csv("category_sales_trends_top50.csv", index=False)

In [24]:
# --- AGGREGATE BY CATEGORY + SUB-DEPARTMENT ---
subdept_trends = (
    top_50_products.groupby(['category', 'category_description',
                             'sub_department', 'sub_department_description'], as_index=False)
    .agg(total_sales=('total_sales', 'sum'),
         total_qty=('total_qty', 'sum'),
         distinct_products=('product_code', 'nunique'))
)

In [25]:
# --- SORT RESULTS ---
subdept_trends = subdept_trends.sort_values(by='total_sales', ascending=False)

In [26]:
subdept_trends.head(10)

,category,category_description,sub_department,sub_department_description,total_sales,total_qty,distinct_products
230,999996.0,DELI MISC,13.0,Cold Deli,65114.01,16066.0,77
145,90201.0,Fresh Meat Ground Beef,21.0,Meat by #,44595.75,3943.0,10
41,36099.0,MISC WATER SOFT DRINKS MIXES,45.0,Soft Drinks,36625.70,8915.0,30
226,999995.0,DELI HOT MISC,12.0,Hot Deli,28845.40,3413.0,13
201,220003.0,MILK,24.0,Wic Dairy,18998.33,5024.0,9
31,33022.0,POTATO CHIP - BAGGED,1.0,Grocery,16531.25,4023.0,30
155,90401.0,Fresh Meat Chicken,21.0,Meat by #,14845.44,1687.0,12
175,91601.0,Smoked Meat Bacon,9.0,Meat,13520.37,2816.0,19
195,130000.0,PLANTS AND FLOWERS,29.0,Floral,9850.32,1561.0,10
0,1006.0,BABY FORMULA/ELECTROLYTE SOLUT,23.0,Wic Grocery,9405.93,382.0,7


In [27]:
# --- OPTIONAL EXPORT ---
subdept_trends.to_csv("category_subdept_sales_top50.csv", index=False)

In [ ]:

# --- GROUP BY Subdept > Category ---
stacked_summary = (
    top_50_products.groupby(['sub_department_description', 'category_description'], as_index=True)
    .agg(
        total_sales=('total_sales', 'sum'),
        total_qty=('total_qty', 'sum'),
        total_cost=('total_cost', 'sum'),
        distinct_products=('product_code', 'nunique')
    )
)

In [55]:
print(stacked_summary.head(10))

                                                     total_sales  total_qty  \
sub_department_description category_description                               
BlackHawk NonTaxable       BH GIFT CARDS                  850.00       14.0   
Cold Deli                  BAKERY ITEMS                  1288.48      225.0   
                           DELI HOT MISC                  348.80       40.0   
                           DELI ITEMS - MEAT DEPT        3724.85      402.0   
                           DELI MISC                    65114.01    16066.0   
                           DELI SALADS AND DESSERTS      4022.83      625.0   
                           DONUTS                        1610.35      185.0   
                           MISC DELI ITEMS                888.20      140.0   
                           T/S BREADS/ROLLS               642.78       72.0   
                           T/S CAKES                     3041.93      341.0   

                                                   

In [56]:
# --- EXPORT OR DISPLAY ---
stacked_summary.to_csv("stacked_subdept_category_summary.csv")